# Starting with a real robot

The BotOp interface should work equally for a simulated and real robot. Note that currently only a locally compiled robotic package supports connecting to a Franka robot and Realsense camera. Here some first cautious steps to get started with a real robot.

### Basic Checklist for operating a Franka
For a full franka setup description see https://frankaemika.github.io/docs/getting_started.html. Here just a minimal checklist for operating a Franka:
* Is the a real time kernel running? Check `uname -r`. The kernel name should end with -rt*
* Is the user in `dialout` and `realtime` group? Check with `groups`. If not, add with `sudo usermod -a -G realtime <username>` and `sudo usermod -a -G dialout <username>`, and log out and in (or reboot) and check again.
* Is the robot ready? E.g. at https://172.16.0.2/desk/. Are joints unlocked and status Ready?
* Test the console tool `ry-bot -real -up`. (See `ry-bot -h` for other test commands, e.g. homing.) Potential issues:
   * Stalls when connecting to Franka - are the Franka IP addresses ({"172.16.0.2", "172.17.0.2"} in franka.cpp) correct?
   * Franka not Ready or can't move - check as above?
   * Error message "== ERROR:franka.cpp:init:356(-2) not implemented with this compiler options". Do you use the pypi package instead of a locally compiled package? Did you enable USE_LIBFRANKA with cmake? Did you `make install`?
   * When executing the cell below, check whether `print(ry.compiled())` prints the correct compile time.

## First robot motion, camera & pcl, and gripper motion

In [1]:
import robotic as ry
import numpy as np
import time
print(ry.compiled())

compile time: Oct 25 2023 23:46:34


Also for a real robot we first load the configuration and maintain our own workspace configuration.

In [3]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))
pcl = C.addFrame('pcl', 'cameraWrist')
C.view(False, 'this is your workspace data structure C -- NOT THE SIMULTATION')

0

Now we start botop with real=True. By passing the model configuration C, the system knows which and how many robots there should be an tries to connect with them.

In [4]:
# True = real robot!!
bot = ry.BotOp(C, useRealRobot=True)

-- bot.cpp:BotOp:38(0) -- blocking useRealRobot -- 


If that failed, you could `ry.params_print()` to see which global parameters were used and whether you should change them. See also the checklist above. Or continue with sim from here one:

In [5]:
if bot.get_t()==0: #if the above failed, use a sim...
    del bot
    bot = ry.BotOp(C, useRealRobot=False)

A first **motion**:

In [6]:
q = bot.get_qHome()
q[1] = q[1] + .2

bot.moveTo(q)
bot.wait(C)
print('first motion done')

bot.moveTo(bot.get_qHome())
bot.wait(C)

first motion done


0

Grabbing a **camera image & pcl**, adding the pcl to the workspace C:

In [7]:
pcl = C.getFrame("pcl")
pcl.setShape(ry.ST.pointCloud, [2]) # the size here is pixel size for display
bot.sync(C)

while bot.getKeyPressed()!=ord('q'):
    image, depth, points = bot.getImageDepthPcl("cameraWrist")
    pcl.setPointCloud(points, image)
    pcl.setColor([1,0,0])
    bot.sync(C, .1)

-- WARNING:frame.cpp:glDraw:1681(-1) trying to draw empty mesh (shape type:pointCloud)


Closing & opening the **gripper**:

In [8]:
#slow close
bot.gripperMove(ry._left, width=.0, speed=.1)

while not bot.gripperDone(ry._left):
    bot.sync(C)

#fast open
bot.gripperMove(ry._left, width=.08, speed=1.)

while not bot.gripperDone(ry._left):
    bot.sync(C)

Always shut down the robot properly by destroying the handle:

In [9]:
del bot
del C

-- bot.cpp:~BotOp:118(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:56(0) shutting down SimThread
-- simulation.cpp:~Simulation:148(0) shutting down Simulation


## Advanced: Compliance & Force/Torque feedback

In [10]:
import robotic as ry
import numpy as np
import time

In [11]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C.view(False, 'this is your workspace data structure C -- NOT THE SIMULTATION')

0

In [12]:
# True = real robot!!
bot = ry.BotOp(C, True)

-- bot.cpp:BotOp:38(0) -- blocking useRealRobot -- 


After opening the robot, it is holding its position. Try moving it you can feel the gains.

In [13]:
C.view(True, 'floating=False, damping=True -- Try moving the robot by hand!\n-- press any key to continue --')

0

We can let it float (=setting the reference always to q_current) and turn off the damping, which makes the robot move more freely:

In [14]:
bot.hold(floating=True, damping=False)
C.view(True, 'floating=True, damping=False -- Try moving the robot by hand!\n-- press any key to continue --')

0

We can also float with daming:

In [15]:
bot.hold(floating=True, damping=True)
C.view(True, 'floating=True, damping=True -- Try moving the robot by hand!\n-- press any key to continue --')

0

The `hold` methods above might be useful for kinestetic teaching or so (you can always keep C sync'ed and compute any features while moving the robot).

But for autonomous compliant manipulation we want to follow a reference and impose compliance in following this reference *along some task space dimensions only*. I.e., a task space compliance.

In [16]:
bot.moveTo(bot.get_qHome(), 1.)
bot.wait(C)

while bot.getKeyPressed()!=ord('q'):
    bot.sync(C, .1)
    y, J = C.eval(ry.FS.position, ["l_gripper"], [[1,0,0]])
    bot.setCompliance(J, 1.)
    print(' direct:', J @ bot.get_tauExternal(),
          ' pseudoInv:', np.linalg.pinv(J.T, rcond=1e-3) @ bot.get_tauExternal())

bot.setCompliance([], 0.)

 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [nan]  pseudoInv: [nan]
 direct: [

In [17]:
bot.home(C)

In [18]:
del bot
del C

-- bot.cpp:~BotOp:118(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:56(0) shutting down SimThread
-- simulation.cpp:~Simulation:148(0) shutting down Simulation


## Testing the depth camera

The following grabs an attached realsense camera, which in our setup is attached by default to the wrist. The robot is put in floating mode, so that you can move the camera. At the same time, the point cloud is attached in the model configuration to the camera frame -- and forward kinematics displays it in world coordinates. That way you can visually see how well you configuration model is calibrated to your real world -- and how noisy the depth camera really is.

In [19]:
import robotic as ry
import numpy as np
import time

In [20]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C.view(False, 'this is your workspace data structure C -- NOT THE SIMULTATION')

0

In [21]:
# True = real robot!!
bot = ry.BotOp(C, False)

In [22]:
bot.hold(floating=True, damping=False)

In [23]:
pclFrame = C.addFrame('pcl', 'cameraWrist')

while bot.getKeyPressed()!=ord('q'):
    rgb, depth, points = bot.getImageDepthPcl('cameraWrist')
    pclFrame.setPointCloud(points, rgb)
    
    bot.sync(C, .1)

In [24]:
del bot

-- bot.cpp:~BotOp:118(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:56(0) shutting down SimThread
-- simulation.cpp:~Simulation:148(0) shutting down Simulation
